<a href="https://colab.research.google.com/github/theostoican/MastersThesis/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from copy import deepcopy
import csv
import torch.nn as nn
import torch
from torch.autograd import Variable

!pip install nlopt
import nlopt
from numpy import *
import numpy as np

# Various modelling parameters

In [29]:
#N is batch size; D_in is input dimension;
#H is the dimension of the hidden layer; D_out is output dimension.
N, D_in, H_teacher, H_student, D_out = 1, 2, 4, 5, 1
num_experiments = 1

# Dataset creation

In [30]:
def construct_dataset():
  data = []
  for y in np.arange(-5, 5.1, .25):
    for x in np.arange(-5, 5.1, .25):
      data.append([x, y])
  return data

data = torch.DoubleTensor(construct_dataset()) 
print(len(construct_dataset()))

1681


# Models

In [31]:
class TeacherNetwork(nn.Module):
  def __init__(self, D_in, H, D_out):
    """
    In the constructor we instantiate two nn.Linear modules and assign them as
    member variables.

    D_in: input dimension
    H: dimension of hidden layer
    D_out: output dimension of the first layer
    """
    super(TeacherNetwork, self).__init__()
    self.linear1 = nn.Linear(D_in, H, bias=False) 
    self.linear2 = nn.Linear(H, D_out, bias=False)
    self.linear1.weight = torch.nn.Parameter(torch.transpose(torch.DoubleTensor([[0.6, -0.5, -0.2, 0.1], [0.5, 0.5, -0.6, -0.6]]), 0, 1))
    self.linear2.weight = torch.nn.Parameter(torch.transpose(torch.DoubleTensor([[1], [-1], [1], [-1]]), 0, 1))
  def forward(self, x):
    """
    In the forward function we accept a Variable of input data and we must
    return a Variable of output data. We can use Modules defined in the
    constructor as well as arbitrary operators on Variables.
    """
    h_sigmoid = torch.sigmoid(self.linear1(x))
    y_pred = self.linear2(h_sigmoid)
    return y_pred

In [32]:
class StudentNetwork(nn.Module):
  def __init__(self, D_in, H, D_out):
    """
    In the constructor we instantiate two nn.Linear modules and assign them as
    member variables.

    D_in: input dimension
    H: dimension of hidden layer
    D_out: output dimension of the first layer
    """
    super(StudentNetwork, self).__init__()
    self.linear1 = nn.Linear(D_in, H, bias=False).double()
    self.linear2 = nn.Linear(H, D_out, bias=False).double()
    nn.init.xavier_uniform_(self.linear1.weight)
    nn.init.xavier_uniform_(self.linear2.weight)
  def forward(self, x):
    """
    In the forward function we accept a Variable of input data and we must
    return a Variable of output data. We can use Modules defined in the
    constructor as well as arbitrary operators on Variables.
    """
    h_sigmoid = torch.sigmoid(self.linear1(x))
    y_pred = self.linear2(h_sigmoid)
    return y_pred

In [33]:
class DummyNetwork(nn.Module):
  def __init__(self, D_in, H, D_out, w_in, w_out):
    """
    In the constructor we instantiate two nn.Linear modules and assign them as
    member variables.

    D_in: input dimension
    H: dimension of hidden layer
    D_out: output dimension of the first layer
    """
    super(DummyNetwork, self).__init__()
    self.linear1 = nn.Linear(D_in, H, bias=False).double()
    self.linear2 = nn.Linear(H, D_out, bias=False).double()
    self.linear1.weight = torch.nn.Parameter(w_in)
    self.linear2.weight = torch.nn.Parameter(w_out)
  def forward(self, x):
    """
    In the forward function we accept a Variable of input data and we must
    return a Variable of output data. We can use Modules defined in the
    constructor as well as arbitrary operators on Variables.
    """
    h_sigmoid = torch.sigmoid(self.linear1(x))
    y_pred = self.linear2(h_sigmoid)
    return y_pred

# Generation of the labels based on the teacher model.

In [34]:
teacher_model = TeacherNetwork(D_in, H_teacher, D_out)
y_labels = teacher_model(data).detach()

# Training

Second order optimization helper method.

In [35]:
def loss_obj(weights, grad):
  w_in = weights[0 : H_student * 2]
  w_out = weights[H_student * 2 :]

  w_in_torch_format = []
  for i in range(H_student):
    w_in_torch_format.append([w_in[2 * i], w_in[2 * i + 1]])
  w_in_torch_format = torch.DoubleTensor(w_in_torch_format)

  w_out_torch_format = torch.DoubleTensor([w_out])

  dummy_model = DummyNetwork(D_in, H_student, D_out, w_in_torch_format, w_out_torch_format)

  loss_val = nn.MSELoss()(dummy_model(data), y_labels)
  
  if grad.size > 0:
    loss_grad = torch.autograd.grad(loss_val, dummy_model.parameters(), create_graph=True)
    gradients = loss_grad[0].reshape(H_student * 2).detach().numpy()
    gradients = np.append(gradients, loss_grad[1][0].detach().numpy())
    grad[:] = gradients

  return loss_val.item()

In [36]:
def train(model, x, y_labels, N = 10, Ninner = 10 ** 5, Nstart = 10,
          maxtime = 3600, nlopt_threshold = 1e-7,
          collect_history = True):
  optimizer = torch.optim.Adam(model.parameters())
  for param_group in optimizer.param_groups:
        lr = param_group['lr']

  loss_fn = nn.MSELoss()
  loss_vals = []
  trace = []
  if collect_history:
    trace.append((deepcopy(model.linear1.weight.data.detach().numpy()),
                  deepcopy(model.linear2.weight.data.detach().numpy())))
  for i in range(1, N + 1):
    loss_tmp = []
    for j in range(1, Ninner + 1):
      y = model(x)
      loss = loss_fn(y, y_labels)
      loss_grad = torch.autograd.grad(loss, model.parameters(), retain_graph=True)
      loss_tmp.append(loss.item())
      optimizer.zero_grad()
      loss.backward(retain_graph=True)
      optimizer.step()
      if i == 1 and (j % Nstart == 0) and j < Ninner:
        loss_vals.append(np.mean(loss_tmp[j - Nstart : j]))
        if collect_history:
          trace.append((deepcopy(model.linear1.weight.data.detach().numpy()),
                        deepcopy(model.linear2.weight.data.detach().numpy())))
    loss_vals.append(np.mean(loss_tmp))
    if collect_history:
      trace.append((deepcopy(model.linear1.weight.data.detach().numpy()),
                    deepcopy(model.linear2.weight.data.detach().numpy())))
    cnt = 0
    for g in loss_grad:
        g_vector = g.contiguous().view(-1) if cnt == 0 else torch.cat([g_vector, g.contiguous().view(-1)])
        cnt = 1
    print("Iteration: %d, loss: %s, gradient norm: %s" % (Ninner * i, np.mean(loss_tmp), torch.norm(g_vector)))
    
    # Adjust the learning rate.
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr / (1 + i)
    
    # stopping criterion
    if np.mean(loss_tmp) < nlopt_threshold or i == N:
        w_in = None
        for w in trace[-1][0]:
          if w_in is None:
            w_in = w
          else:
            w_in = np.append(w_in, w) 
        w_out = trace[-1][1][0]
        weights = np.append(w_in, w_out)

        opt = nlopt.opt(nlopt.LD_SLSQP, len(weights))
        opt.set_lower_bounds([w - 10 for w in weights])
        opt.set_upper_bounds([w + 10 for w in weights])
        opt.set_min_objective(loss_obj)
        opt.set_maxtime(maxtime)
        final_weights = opt.optimize(weights)
        return loss_vals, trace, final_weights

# Hessian evaluation

In [37]:
def eval_hessian(loss_grad, model):
  cnt = 0
  for g in loss_grad:
      g_vector = g.contiguous().view(-1) if cnt == 0 else torch.cat([g_vector, g.contiguous().view(-1)])
      cnt = 1
  grad_norm = torch.norm(g_vector)
  l = g_vector.size(0)
  hessian = torch.zeros((l, l), dtype = torch.float64)
  for idx in range(l):
      grad2rd = torch.autograd.grad(g_vector[idx], model.parameters(), create_graph=True)
      cnt = 0
      for g in grad2rd: 
          g2 = g.contiguous().view(-1) if cnt == 0 else torch.cat([g2, g.contiguous().view(-1)])
          cnt = 1
      hessian[idx] = g2
  # Symmetrize the Hessian.
  hessian = (hessian + hessian.T) / 2
  return grad_norm.detach().numpy(), hessian.detach().numpy()

In [38]:
file_experiment_header = ['loss', 'gradient norm', 'smallest eigenvalue', 'student size']

for i in range(0, H_student):
  file_experiment_header.append('neuron_' + str(i) + '_traj_x')
  file_experiment_header.append('neuron_' + str(i) + '_traj_y')
  file_experiment_header.append('neuron_' + str(i) + '_a')

file_experiment_header.append('teacher_neurons_x')
file_experiment_header.append('teacher_neurons_y')


file_experiment_data = open('experiments_data.csv', 'w')
writer = csv.writer(file_experiment_data)
writer.writerow(file_experiment_header)

for num_experiment in range(0, num_experiments):
  student_model = StudentNetwork(D_in, H_student, D_out)
  loss_vals, trace, final_weights = train(student_model, data, y_labels)

  w_in = torch.DoubleTensor(final_weights[0 : 2 * H_student].reshape(H_student, 2))
  w_out = torch.DoubleTensor([final_weights[2 * H_student :]])
  dummy_model = DummyNetwork(D_in, H_student, D_out, w_in, w_out)
  last_loss_val = nn.MSELoss()(dummy_model(data), y_labels).item()
  loss_grad = torch.autograd.grad(nn.MSELoss()(dummy_model(data), y_labels), dummy_model.parameters(), create_graph=True)
  grad_norm, hessian = eval_hessian(loss_grad, dummy_model)
  smallest_eigenvalue = np.min(np.linalg.eigvals(hessian))

  row = [last_loss_val, grad_norm, smallest_eigenvalue, H_student]

  for i in range(0, H_student):
    neuron_w_x = []
    neuron_w_y = []
    neuron_a = []
    for (inp_weights, out_weights) in trace:
      neuron_w_x.append(inp_weights[i][0])
      neuron_w_y.append(inp_weights[i][1])
      neuron_a.append(out_weights[0][i])
    neuron_w_x.append(w_in[i][0])
    neuron_w_y.append(w_in[i][1])
    neuron_a.append(w_out[0][i])
    row.append(neuron_w_x)
    row.append(neuron_w_y)
    row.append(neuron_a[-1])

  teacher_neurons_x = [0.6, -0.5, -0.2, 0.1]
  teacher_neurons_y = [0.5, 0.5, -0.6, -0.6]
  row.append(teacher_neurons_x)
  row.append(teacher_neurons_y)

  writer.writerow(row)

file_experiment_data.close()

Iteration: 100, loss: 0.5757672203405921, gradient norm: tensor(1.3830, dtype=torch.float64)
Iteration: 200, loss: 0.33436703097172, gradient norm: tensor(1.1365, dtype=torch.float64)
Iteration: 300, loss: 0.23932154961015215, gradient norm: tensor(0.9846, dtype=torch.float64)
Iteration: 400, loss: 0.18522761692665035, gradient norm: tensor(0.8759, dtype=torch.float64)
Iteration: 500, loss: 0.1496795581572889, gradient norm: tensor(0.7920, dtype=torch.float64)
Iteration: 600, loss: 0.12437412398564046, gradient norm: tensor(0.7242, dtype=torch.float64)
Iteration: 700, loss: 0.10540044240765488, gradient norm: tensor(0.6675, dtype=torch.float64)
Iteration: 800, loss: 0.09064195419591424, gradient norm: tensor(0.6191, dtype=torch.float64)
Iteration: 900, loss: 0.07884246339295255, gradient norm: tensor(0.5769, dtype=torch.float64)
Iteration: 1000, loss: 0.06920522402823324, gradient norm: tensor(0.5397, dtype=torch.float64)


# Plotting

In [39]:
# teacher_neurons_x = [0.6, -0.5, -0.2, 0.1]
# teacher_neurons_y = [0.5, 0.5, -0.6, -0.6]

In [40]:

# neuron_0_w_x = []
# neuron_0_w_y = []
# neuron_0_a = []

# neuron_1_w_x = []
# neuron_1_w_y = []
# neuron_1_a = []

# neuron_2_w_x = []
# neuron_2_w_y = []
# neuron_2_a = []

# neuron_3_w_x = []
# neuron_3_w_y = []
# neuron_3_a = []

# neuron_4_w_x = []
# neuron_4_w_y = []
# neuron_4_a = []

# for (inp_weights, out_weights) in trace:
#   neuron_0_w_x.append(inp_weights[0][0])
#   neuron_0_w_y.append(inp_weights[0][1])
#   neuron_0_a.append(out_weights[0][0])

#   neuron_1_w_x.append(inp_weights[1][0])
#   neuron_1_w_y.append(inp_weights[1][1])
#   neuron_1_a.append(out_weights[0][1])

#   neuron_2_w_x.append(inp_weights[2][0])
#   neuron_2_w_y.append(inp_weights[2][1])
#   neuron_2_a.append(out_weights[0][2])

#   neuron_3_w_x.append(inp_weights[3][0])
#   neuron_3_w_y.append(inp_weights[3][1])
#   neuron_3_a.append(out_weights[0][3])

#   neuron_4_w_x.append(inp_weights[4][0])
#   neuron_4_w_y.append(inp_weights[4][1])
#   neuron_4_a.append(out_weights[0][4])

# plt.plot(neuron_0_w_x, neuron_0_w_y)
# plt.plot(neuron_1_w_x, neuron_1_w_y)
# plt.plot(neuron_2_w_x, neuron_2_w_y)
# plt.plot(neuron_3_w_x, neuron_3_w_y)
# plt.plot(neuron_4_w_x, neuron_4_w_y)

# plt.scatter(teacher_neurons_x, teacher_neurons_y, marker="*")

# outgoing_weights = [neuron_0_a[-1], neuron_1_a[-1], neuron_2_a[-1], neuron_3_a[-1], neuron_4_a[-1]]
# plt.scatter([neuron_0_w_x[-1], neuron_1_w_x[-1], neuron_2_w_x[-1], neuron_3_w_x[-1], neuron_4_w_x[-1]],
#             [neuron_0_w_y[-1], neuron_1_w_y[-1], neuron_2_w_y[-1], neuron_3_w_y[-1], neuron_4_w_y[-1]],
#             c = outgoing_weights,
#             cmap=matplotlib.cm.jet)
# plt.colorbar()

# # Teacher's neurons
# #[0.6, -0.5, -0.2, 0.1],
# #[0.5, 0.5, -0.6, -0.6],

# Checking for local minima

In [41]:
# class DummyNetwork(nn.Module):
#   def __init__(self, D_in, H, D_out, w_in, w_out):
#     """
#     In the constructor we instantiate two nn.Linear modules and assign them as
#     member variables.

#     D_in: input dimension
#     H: dimension of hidden layer
#     D_out: output dimension of the first layer
#     """
#     super(DummyNetwork, self).__init__()
#     self.linear1 = nn.Linear(D_in, H, bias=False) 
#     self.linear2 = nn.Linear(H, D_out, bias=False)
#     self.linear1.weight = torch.nn.Parameter(w_in)
#     self.linear2.weight = torch.nn.Parameter(w_out)
#   def forward(self, x):
#     """
#     In the forward function we accept a Variable of input data and we must
#     return a Variable of output data. We can use Modules defined in the
#     constructor as well as arbitrary operators on Variables.
#     """
#     h_sigmoid = torch.sigmoid(self.linear1(x))
#     y_pred = self.linear2(h_sigmoid)
#     return y_pred

In [42]:
# print (trace[-1])
# def loss_reducer(w_in, w_out):
#   dummy_model = DummyNetwork(D_in, H_teacher, D_out, w_in, w_out)
#   return nn.MSELoss()(dummy_model(data), y_labels)
  
# print(loss_reducer(torch.Tensor(trace[-1][0]), torch.Tensor(trace[-1][1])))
# H = torch.autograd.functional.hessian(loss_reducer, (torch.Tensor(trace[-1][0]), torch.Tensor(trace[-1][1])))
# eval Hessian matrix
# def eval_hessian(loss_grad, model):
#     cnt = 0
#     for g in loss_grad:
#         g_vector = g.contiguous().view(-1) if cnt == 0 else torch.cat([g_vector, g.contiguous().view(-1)])
#         cnt = 1
#     print(torch.norm(g_vector))
#     l = g_vector.size(0)
#     hessian = torch.zeros(l, l)
#     for idx in range(l):
#         grad2rd = torch.autograd.grad(g_vector[idx], model.parameters(), create_graph=True)
#         cnt = 0
#         for g in grad2rd: 
#             g2 = g.contiguous().view(-1) if cnt == 0 else torch.cat([g2, g.contiguous().view(-1)])
#             cnt = 1
#         hessian[idx] = g2
#     return hessian.cpu().data.numpy()